In [94]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pickle
import nltk
nltk.download('punkt')
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from string import punctuation
import numpy as np
import re
from sklearn import feature_extraction
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from nltk.stem import PorterStemmer
from nltk.corpus import words

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/austinkrause/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [97]:
import os

In [103]:
#load in our xgb classification model
xgb = pickle.load(open("Models/xgboost_model.sav", "rb"))

FileNotFoundError: [Errno 2] No such file or directory: 'Models/xgboost_model.sav'

In [4]:
def create_soup(url):
    facts = requests.get(url)
    soup = BeautifulSoup(facts.content, 'html.parser')
    return soup

In [77]:
def get_article_text(url):
    #url = input('Enter a url: \n')
    soup = create_soup(url)
    
    text = []
    for i in soup.find_all('p'):
        text.append(i.get_text())

    full_text = ' '.join(text).replace('\xa0', '')
#     word_count_ = full_text.count(' ')
#     gensim_summary = summarize(full_text, word_count = word_count_/10).strip('. ').replace('.\n', '')
    
#     print('\n Full Text:\n', full_text, '\n\n')
#     print('Extractive Summary: \n', gensim_summary)
    return full_text

In [234]:
#get_article_text('https://news.yahoo.com/cambodia-islands-under-threat-woman-trying-save-them-144633896.html')



In [79]:
#set stop words
stops = list(set(stopwords.words('english'))) + list(punctuation)

In [80]:
stemmer = PorterStemmer()

In [81]:
#helper function to remove stop words
def remove_stops(text):
    text_no_stops = []
    for i in text:
        i = i.lower()
        if i not in stops:
            if len(i) == 1:
                pass
            else:
                text_no_stops.append(i)
        else:
            pass
    return text_no_stops

In [82]:
#helper function to lemmatize text
def lemmatize_text(text):
    lemmatized = []
    for word in text:
        lemmatized.append(lemmatizer.lemmatize(word))
    return lemmatized

In [83]:
#create helper function to stem each word in a list and concat the list
def stem_list(lst):
    stemmed_list = []
    for i in lst:
        stemmed_list.append(stemmer.stem(i))
    stem_string = ' '.join(stemmed_list[:100])
    return stem_string

In [84]:
#function to perform lemmatization, stemming, and final preprocessing
def xgb_text_prep(sample_text):
    tokens = word_tokenize(str(sample_text), language = 'en')
    no_stops = remove_stops(tokens)
    stemmed = stem_list(no_stops)
    #stemmed = stemmed.replace('\\n', '')
    return [stemmed]
    #return no_stops

In [86]:
#function for xgboost prdiction
def get_xgb_prediction(text):
    preprocess = text_prep([text])
    return xgb.predict(preprocess)[0]

In [87]:
#find cosine similarities between sentences
def find_similarities(text):
    #tokenize sentences
    sentences = sent_tokenize(text, language = 'en')
    
    #vectorize sentences
    vectorizer = TfidfVectorizer(stop_words = stops)
    trsfm=vectorizer.fit_transform(sentences)
    #creat df for article
    text_df = pd.DataFrame(trsfm.toarray(),columns=vectorizer.get_feature_names(),index=sentences)
    
    #declare how many sentences to use in summary
    num_sentences = text_df.shape[0]
    num_summary_sentences = int(np.ceil(num_sentences**.5))
        
    #find cosine similarity for all sentences
    similarities = cosine_similarity(trsfm, trsfm)
    #print(similarities)
    #create list to hold avg cosine similarities for each sentence
    avgs = []
    for i in similarities:
        avgs.append(i.mean())
     
    #find index values of the sentences to be used for summary
    top_idx = np.argsort(avgs)[-num_summary_sentences:]
    
    return top_idx

In [88]:
def build_summary(text):
    sents_for_sum = find_similarities(text)
    sort = sorted(sents_for_sum)
    print('\n Sentences selected:', sort)
    
    sent_list = sent_tokenize(text)
    print('\n Total sentences:', len(sent_list))
    
    sents = []
    for i in sort:
        sents.append(sent_list[i].replace('\n', ''))
    
    summary = ' '.join(sents)
    return summary

In [90]:
def xgb_and_summary():
    url = input('Enter a url: \n')
    text = get_article_text(url)
    print('\n')
    xgb_preprocess = xgb_text_prep(text)
    print('This article was placed into cluster:', xgb.predict(xgb_preprocess)[0])
    
    summary = build_summary(text)
    return summary

TEST ARTICLE 1 --- Supreme Court --- returns cluster 7
https://news.yahoo.com/u-supreme-court-spurns-virginia-143413283.html;_ylt=AwrC0COK7wddIj8AGCPQtDMD;_ylu=X3oDMTEydGhybm9tBGNvbG8DYmYxBHBvcwMxBHZ0aWQDQjc2MDlfMQRzZWMDc3I-

TEST ARTICLE 2 --- Donald Trump --- returns cluster 1
https://news.yahoo.com/trump-says-d-foreigners-offer-233911079.html;_ylt=AwrC1C7Y8AddADIAoS_QtDMD;_ylu=X3oDMTEyaTE5czNyBGNvbG8DYmYxBHBvcwM0BHZ0aWQDQjc2MDlfMQRzZWMDc3I-

TEST ARTICLE 3 --- Syria --- returns cluster 11
https://news.yahoo.com/turkish-outpost-syria-shelled-syrian-080926968.html;_ylt=AwrC0wwt8gdd2WgAkwHQtDMD;_ylu=X3oDMTEydGhybm9tBGNvbG8DYmYxBHBvcwMxBHZ0aWQDQjc2MDlfMQRzZWMDc3I-

TEST ARTICLE 4 --- Birth Control Health Insurance --- returns cluster 3
https://news.yahoo.com/democrat-warren-wants-7-billion-134454649.html;_ylt=AwrC1DGZ8gddwlsAJwLQtDMD;_ylu=X3oDMTEydDBqZzZmBGNvbG8DYmYxBHBvcwMyBHZ0aWQDQjc2MDlfMQRzZWMDc3I-

TEST ARTICLE 5 --- Bernie Sanders --- returns cluster 3
https://news.yahoo.com/sanders-outline-democratic-socialism-means-045302693.html;_ylt=AwrC0CMD4whdTSoALAHQtDMD;_ylu=X3oDMTEydDBqZzZmBGNvbG8DYmYxBHBvcwMyBHZ0aWQDQjc2MDlfMQRzZWMDc3I-

In [91]:
xgb_and_summary()

Enter a url: 
https://www.nytimes.com/2019/06/14/books/review/podcast-underland-robert-macfarlane-disappearing-earth-julia-phillips.html




NameError: name 'xgb' is not defined

In [75]:
def get_ny_articles(url):
    page = create_soup(url)
    a= test_.find_all('a', href = True)
    
    ny_times_links = []
    for a in test_.find_all('a', href=True):
        ny_times_links.append(a['href'])
    
    ny_times_articles = []
    for i in ny_times_links:
        if i[0] == '/' and len(i)>2 and i.endswith('Container') == False:
            ny_times_articles.append(i)
    
    article_set = set(ny_times_articles)
    return article_set

In [76]:
get_ny_articles('https://www.nytimes.com/')

{'/2019/06/14/books/review/podcast-underland-robert-macfarlane-disappearing-earth-julia-phillips.html',
 '/2019/06/14/the-weekly/trump-immigration-border-separation-family.html',
 '/2019/06/18/arts/music/taylor-swift-you-need-to-calm-down-video.html',
 '/2019/06/18/books/pakistan-darra-adam-khel-library.html',
 '/2019/06/18/briefing/patrick-shanahan-orlando-taylor-swift.html',
 '/2019/06/18/business/ecb-mario-draghi-stimulus.html',
 '/2019/06/18/business/economy/global-economy-trade-war.html',
 '/2019/06/18/business/japan-work-overtime-tv-show.html',
 '/2019/06/18/dining/van-da-restaurant-review.html',
 '/2019/06/18/nyregion/central-park-five-trump.html',
 '/2019/06/18/nyregion/greenhouse-gases-ny.html',
 '/2019/06/18/opinion/androgyny-wnba-fashion.html',
 '/2019/06/18/opinion/guatemala-election.html',
 '/2019/06/18/opinion/morsi-death-egypt.html',
 '/2019/06/18/opinion/pete-chasten-buttigieg.html',
 '/2019/06/18/opinion/pregnant-athlete-sponsor-climber.html',
 '/2019/06/18/opinion/rel

In [69]:
xgb_an

In [78]:
get_article_text('https://www.nytimes.com/2019/06/14/books/review/podcast-underland-robert-macfarlane-disappearing-earth-julia-phillips.html')

'Advertisement Subscribe: iTunes | Google Play Music | How to Listen The acclaimed British writer Robert Macfarlane, who is not altogether comfortable with the term “nature writer,” has produced books about mountains, very long walks through Britain and the language we use to describe landscapes. In his new book, “Underland,” Macfarlane goes underground. He goes beneath forest floors, and into sea caves and sinkholes, among other subterranean adventures, to probe the secrets of man’s often malign influence on the earth. “It’s pretty unnatural,” Macfarlane says on this week’s podcast. “There are reasons why very little living goes on down there. Before I went anywhere, I got a friend to take me — I just said, look, take me underground for a day in Britain, let’s go somewhere scary. I need to see if I can be in these places. And I absolutely loved it. He described me later as a ‘rat in a drain pipe.’ I think that was a compliment, I’m not sure.”  Julia Phillips visits the podcast this we